L'algorithme CART est un modèle **d'arbre de décision** destinée à la classification ([disponible ici](https://github.com/titigmr/ML/blob/master/CART.ipynb)). Il est construit sur le principe de « série de questions » (appelées *split*, *noeuds* ou encore *embranchements*). Ces questions conduisent sucessivement à une feuille de l'arbre (c'est à dire quand un split n'est plus possible). Pour y arriver, l'algorithme utilise le concept d'impureté de Gini. Cette métrique indique à quel point un noeud est pur (autrement dit à quel point une seule modalité apparait sur un embranchement). 

Pour ce faire, l'arbre de décision forme des noeuds en recherchant pour chacune des variables le split binaire optimal (qu'on appelle *thresholds*). L'objectif est de maximiser l'indice de Gini parmi tous les seuils possibles et retenir le meilleur indice (le plus faible) parmi l'ensemble des variables. 

Pour finir, le modèle s'arrête dès que tous les noeuds sont purs ou lorsqu'il n'est plus possible de séparer un sous-échantillon.

Toutefois, un des grands désavantage de l'arbre de décision est qu'il est très sensible à l'effet de surapprentissage dès que le nombre d'embranchement augmente (la capaciter de bien généraliser sur de nouvelles données). Ce surapprentissage se produit car le modèle est très flexible (**forte variance**) et capture sur les données tout le bruit présent.

De l'autre côté, si nous limitons le nombre d'embranchements, nous limitons certe la variance du modèle (donc le fait que le modèle puisse sur-apprendre) mais en contre-partiele modèle ne sera pas capable de bien prédire car des hypothèses forte sur les données sont avancées.

Au lieu de limiter la profondeur de l'arbre, ce qui réduit la variance et augmente le biais, nous pouvons combiner de nombreux arbres de décision en un seul modèle d'ensemble connu sous le nom de forêt aléatoire.

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()

X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name="y")

# Random Forests

La Forêt aléatoire ou Random Forests est un modèle dont le but est de combiner de nombreux arbres de décision. Toutefois, ce modèle possède deux spécificité. Plutôt que de simplement faire la moyenne/vote de la prédiction de chacun des arbres présent dans la fôret, ce modèle utilise deux concepts clés qui lui donnent le nom aléatoire :

- **Bootstrapping** : Échantillonnage aléatoire des données d'entraînement lors de la création de chaque arbres ; 
- **Sélection aléatoires des features** : Sous-ensembles aléatoires de variables pris en compte lors de la division des nœuds ;

## Bootstraping 

L'objectif du bootstraping est d'utiliser comme données d'entraînmenet un échantillon aléatoire du jeu de données. Les échantillons sont sélectionnés avec remise, appelé *bootstrap*, ce qui signifie que certaines observations seront utilisés plusieurs fois. 

L'idée est qu'en entraînant chaque arbre sur différents échantillons, bien que chaque arbre puisse avoir une variance élevée par rapport à un ensemble particulier de données d'apprentissage, dans l'ensemble, la forêt entière aura une variance plus faible, mais pas au prix d'une augmentation du biais.


In [3]:
def bootstrap(X, y):
    indices = np.random.randint(len(y), size=len(y))
    return X.loc[indices], y.loc[indices]

In [4]:
X_, y_ = bootstrap(X,y)

Au moment de la prédiction, celle-ci est faite en faisant : 

- la moyenne des prédictions de chaque modèle (dans la cas d'une variable quantitative) ;
- le vote parmi la prédition majoritaire pour l'ensemble des modèles (dans le cas d'une variable qualitative).

Cette méthode d'ensemble qui utilise des bootstrap des données, puis de calcul de la moyenne/vote des prédictions est appelé : **Bagging** (abréviation de Bootstrap AGGregatING)

## Random features

L'autre concept principal est qu'un seul sous-ensemble de variables est pris en compte pour diviser chaque nœud dans chaque arbre de décision. Généralement, il est défini comme `sqrt (n_features)` pour de la classification.

In [5]:
def select_features(columns, metric='auto'):
    
    if metric == 'auto':
        n = int(np.sqrt(len(columns)))
        selected_features = np.random.choice(columns, size=n, replace=False)
    
    if metric == 'all':
        selected_features = columns
        
    if type(metric) == int:
        n = metric
        selected_features = np.random.choice(columns, size=n, replace=False)
        
    return selected_features

In [6]:
select_features(X.columns)

array(['sepal length (cm)', 'petal length (cm)'], dtype=object)

## Construction de la fôret : un ensemble d'arbres de décision

La dernière étape consiste à l'implémentation du Random Forest. Dans un premier temps, il faut multiplier les arbre de décision simples [voir algorithme CART](https://github.com/titigmr/ML/blob/master/CART.ipynb). Dans un second temps, lorqu'une prédiction est faite, agréger l'ensemble des prédictions de chacun des arbres et retenir celle qui a le plus de vote.

In [7]:
from CART import DecisionTreeClassif

In [9]:
def generate_forest(X, y, n_estimators=5, target_name=None):

    all_tree = []

    for i in range(n_estimators):
        X_boot, y_boot = bootstrap(X, y)

        X_boot.reset_index(drop=True, inplace=True)
        y_boot.reset_index(drop=True, inplace=True)

        clf = DecisionTreeClassif(max_features='auto')
        clf.fit(X_boot, y_boot, target_name=target_name)

        all_tree.append(clf)

    return all_tree

## Prédiction des données

La dernière étape consiste à agréger les prédictions de chaque arbre de la forêt pour fournir une meilleur prédiction qu'un simple arbre de décision

In [10]:
from scipy.stats import mode

In [17]:
class RandomForestClassif:
    
    
    def __init__(self, max_depth=float('inf'), max_features="auto", n_estimators=5):
        
        self.max_depth = max_depth + 1
        self.max_features = max_features
        self.n_estimators = n_estimators
        
        
    def fit(self, X, y, target_name=None):
        
        X.reset_index(drop=True, inplace=True)
        y.reset_index(drop=True, inplace=True)
        
        all_tree = []

        for i in range(self.n_estimators):
            X_boot, y_boot = self.__bootstrap__(X, y)

            X_boot.reset_index(drop=True, inplace=True)
            y_boot.reset_index(drop=True, inplace=True)

            clf = DecisionTreeClassif(max_features=self.max_features)
            clf.fit(X_boot, y_boot, target_name=target_name)

            all_tree.append(clf)

        self.__all_tree__ = all_tree
    
    def __bootstrap__(self, X, y):
        indices = np.random.randint(len(y), size=len(y))
        return X.loc[indices], y.loc[indices]
    
    
    def predict(self, X):
        return [self._predict_(inputs, X) for inputs in X.index]


    def _predict_(self, inputs, X):
        
        X.reset_index(drop=True, inplace=True)
        
        all_predicts = []
        for tree in self.__all_tree__ :
            node = tree.tree
            
            while node.node.left:
                if X.loc[inputs, node.node.var] < node.node.threshold:
                    node = node.node.left

                else:
                    node = node.node.right

                if node.node is None:
                    break

            all_predicts.append(node.classe_predict)
            
        return mode(all_predicts).mode[0]

On divise l'échantillon en train et test 

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [34]:
clf = RandomForestClassif(max_depth=4)
clf.fit(X_train, y_train, iris.target_names)

In [35]:
sum(clf.predict(X_test) == iris.target_names[y_test]) / len(y_test)

0.98

## Avec sklearn

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [39]:
rdf = RandomForestClassifier(max_depth=4)
rdf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [40]:
sum(rdf.predict(X_test) == y_test) / len(y_test)

0.98

# Références

[Towardsdatascience.com : RandomForest](https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76)